In [1]:
import os
if os.name != 'nt' :
    import resource
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML
from App.common import V, init_globals, loading_bar, myfigures, get_query_list, mp, tabs_out, tabs_html
from App import tab1, tab2, tab3, tab4, tab5, tab6, tab7
import dmyplant2
from dmyplant2 import cred, MyPlant, Engine, cplotdef, get_size
import logging
import shutil

logfilename = 'FSM.log'
if os.path.exists(logfilename):
    shutil.copyfile(logfilename,logfilename[:-4] + '_bak.log')

logging.basicConfig(
    filename=logfilename,
    filemode='w',
    format='%(asctime)s %(levelname)s %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    level=logging.DEBUG
)
logging.info("FSM logging Start")

### Statemachine Engine Analysis ©2022 Dieter Chvatal

In [2]:
#########################################
# tabs
#########################################
class App():
    def __init__(self):
        
        self.tab_objects_list = [
            tab1.Tab(),
            tab2.Tab(),
            tab3.Tab(),
            tab4.Tab(),
            tab5.Tab(),
            tab6.Tab(),
            #tab7.Tab()
        ]

        self.tabs = widgets.Tab()
        self.tabs.observe(self.tabs_change_index, 'selected_index')

        self.set_children()
        self.set_titles()

        self.tab_objects_list[0].selected() # preselect the first tab.

    def show(self):
        display(
            widgets.VBox([
                self.tabs,
                tabs_out,
                tabs_html])
            );

    def set_children(self):
        self.tabs.children = [t.tab for t in self.tab_objects_list]

    def set_titles(self):
        for i,t in enumerate(self.tab_objects_list):
            self.tabs.set_title(i,t.title)

    def tabs_change_index(self, *args):  # the tabs callback
        self.tab_objects_list[self.tabs.selected_index].selected()


app = App()
app.show()

In [3]:
def disp_alwr(rec):
    ll = []
    for m in rec:
        ll.append({
            'datetime':pd.to_datetime(int(m['msg']['timestamp'])*1e6).strftime('%Y-%m-%d %H:%M:%S'),
            'state': m['state'],
            'number': m['msg']['name'],
            'type': 'Alarm' if m['msg']['severity'] == 800 else 'Warning',
            'severity': m['msg']['severity'],
            'message': m['msg']['message']
        })
    if len(rec) > 0:
        display(HTML(pd.DataFrame(ll).to_html(index=False, header=False)))

def display_fmt(df):
    display(df[['starttime'] + V.fsm.results['run2_content']['startstop']]                    
                        .style
                        .hide()
                        .format(
                    precision=2,
                    na_rep='-',
                    formatter={
                        'starttime': "{:%Y-%m-%d %H:%M:%S %z}",
                        'startpreparation': "{:.0f}",                        
                        'starter': "{:.1f}",
                        'speedup': "{:.1f}",
                        'idle': "{:.1f}",
                        'synchronize': "{:.1f}",
                        'loadramp': "{:.0f}",
                        'cumstarttime': "{:.0f}",
                        'targetload': "{:.0f}",
                        'ramprate':"{:.2f}",
                        'maxload': "{:.0f}",
                        'targetoperation': "{:.0f}",
                        'rampdown': "{:.1f}",
                        'coolrun': "{:.1f}",
                        'runout': lambda x: f"{x:0.1f}"
                    }
                ))

In [4]:
rde = V.rdf
rde['AW'] = rde.apply(lambda x: x['A'] + x['W'] > 0, axis=1)
j = 0; k = 0
rowgen = rde.iterrows()
try:
    while True:
        i,row = next(rowgen)
        if row['AW']:
            if i-k > 0:
                display_fmt(V.rdf.iloc[k:i])
            display_fmt(row.to_frame().T)
            disp_alwr(row['alarms'])
            disp_alwr(row['warnings'])
            k = i + 1
except StopIteration:
    pass


In [5]:
if V.fsm:
    pp(V.fsm.results['run2_content'])

In [6]:
if V.fsm:
    df = V.fsm.starts[['alarms','warnings','A','W']]
    dfw = df[df['W']>1]
    [w['msg']['name'] for warnings in  df['warnings'] for w in warnings]

In [7]:
if V.fsm:
    [w['msg']['name'] == '2688' for warnings in  df['warnings'] for w in warnings]

In [8]:
if V.fsm:
    [w['msg']['name'] == '1105' for alarms in  df['alarms'] for w in alarms]

In [9]:
#hasattr(V.fsm, '_messages')

In [10]:
#if V.fsm is not None:
#    V.fsm.act_run, len(V.fsm.starts)

In [11]:
# from App import tab4
# if V.fsm is not None:
#     tab4.update_fig(tab4.sno.value, plotselection=tab4.plotselection.value,plot_range=tab4.time_range.value,fsm=V.fsm, VSC=True)

In [12]:
from pprint import pprint as pp

if os.name != 'nt':
    pass
    #memory = resource.getrusage(resource.RUSAGE_SELF)
    #print(f"Script Memory Usage: {memory.ru_utime:5.1f} MB")

if V.fsm is not None:
    print("\nV.fsm:")
    print("------")
    print(f"{'Size fsm in MB':>40}: {get_size(V.fsm) / (1024.0*1024.0):10.3f} MB")
    keys = list (V.fsm.__dict__.keys())
    for k in keys:
        print(f"{k:>40}: {get_size(V.fsm.__dict__[k]) / (1024.0*1024.0):10.3f} MB")

    print("\n_e:")
    print("--")
    print(f"{'Size fsm._e in MB':>40}: {get_size(V.fsm._e) / (1024.0*1024.0):10.3f} MB")
    sshkeys = list (V.fsm._e.__dict__.keys())
    for sshk in sshkeys:
        print(f"{sshk:>40}: {get_size(V.fsm._e.__dict__[sshk]) / (1024.0*1024.0):10.3f} MB")    

    print("\nstartstopHandler:")
    print("-----------------")
    print(f"{'Size fsm.startstopHandler in MB':>40}: {get_size(V.fsm.startstopHandler) / (1024.0*1024.0):10.3f} MB")
    sshkeys = list (V.fsm.startstopHandler.__dict__.keys())
    for sshk in sshkeys:
        print(f"{sshk:>40}: {get_size(V.fsm.startstopHandler.__dict__[sshk]) / (1024.0*1024.0):10.3f} MB")

In [13]:
if V.fsm is not None:
    print("\nstartstopHandler._states:")
    print("-------------------------")
    print(f"{'Size fsm.startstopHandler._states in MB':>40}: {get_size(V.fsm.startstopHandler._states) / (1024.0*1024.0):10.3f} MB")
    sshkeys = list (V.fsm.startstopHandler._states)
    for sshk in sshkeys:
        print(f"{sshk:>40}: {get_size(V.fsm.startstopHandler._states[sshk]) / (1024.0*1024.0):10.3f} MB")

In [14]:
#V.fsm.save_results('test.dfsm')

In [15]:
#V.fsm._messages

In [16]:
#pd.DataFrame(V.fsm.message_queue[935:937])

In [17]:
#d = V.fsm.message_queue[936:937][0]
#d

In [18]:
#pd.DataFrame([d])

In [19]:
#V.fsm.message_queue[935]

In [20]:
#d=V.fsm.message_queue[936]
#d['assetId'] = 114517
#d

In [21]:
#pd.DataFrame(d, index=[0], dtype=object).T

In [22]:
if V.fsm is not None:
    V.fsm._e._calc_BMEP(4881.0, 1500.0)